<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/Miky_TupeloHoney_yelp_picture_downloaderRandom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#da cambiare:
# folder_id 
import requests
from lxml import html
import shutil
import os
import json

def downloadUser(user_id, counter):

    #Creare la cartella sul Drive dove andranno le foto (in data > face_data > Foto_User > (nomeRistorante)10%(strategia))
    folder_id = '1Y4kVSFpVyLUA6uIAE9pAN12U82KNRZ6-'  # folder ID = CaboFishTaco_10%Random
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id

    try:
        # Find user picture from web page
        page_content = requests.get(url)
        tree = html.fromstring(page_content.content)
        if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) != 0:  # Picture found
            image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

            # View all folders and file in your folder
            fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()

            # Create file on Google Drive
            # param id = ID of Fairness/face_data folder in my Google Drive
            drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})

            # Check if file already exists in Google Drive (prevents duplicates)
            for file in fileList:
                # print('Title: %s, ID: %s' % (file['title'], file['id']))
                if file['title'] == filename:  # The file already exists, then overwrite it
                    fileID = file['id']
                    drive_file = drive.CreateFile({'id': fileID, 'title': filename, 'parents': [{'id': folder_id}]})

            response = requests.get(image_url, stream=True)

            # Create a local copy of user picture
            with open(filename, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            del response

            # Upload user picture on Google Drive
            drive_file.SetContentFile(filename)  # path of local file content
            drive_file.Upload()  # Upload the file.
            # print('title: %s, mimeType: %s' % (drive_file['title'], drive_file['mimeType']))

            # Delete local user pictures
            if os.path.exists(filename):
                os.remove(filename)
            else:
                print("The file does not exist")

    except:
        print("Si è verificato un errore alla riga " + str(counter) + " del file user.json")

In [0]:
#Da cambiare:
# users_restaurant_id, il nome del csv e quante foto prendere (10% del totale delle recensioni)
# file_list mettere l'id della cartella da dove prendere le foto

#10% foto di utenti random per ristorante
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import csv
import random
import pandas as pd

# Connect to Google Account
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nomeRistorante = 'Tupelo Honey'
# DOWNLOAD Restaurant.csv FROM DRIVE
# users_restaurant_id = id del file csv del ristorante (si trova in data > Scraping > Reviews_Relevance > (nomeRistorante)_relevance_desc.csv)
users_restaurant_id = '1aOTHWSNdkjzW7EyyaomdXxgnzjXdqN7S'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_restaurant_id})
download.GetContentFile('%s' % nomeRistorante + '_relevance_desc.csv')

#leggo il csv
restaurant = pd.read_csv('%s' % nomeRistorante + '_relevance_desc.csv')

file_list = drive.ListFile({'q': "'1Y4kVSFpVyLUA6uIAE9pAN12U82KNRZ6-' in parents and trashed=false"}).GetList()
listaVecchia = []
for file1 in file_list:
  id = file1['title']
  listaVecchia.append(id[0:22])

#user_list = lista id utenti che ci sono anche nel dataset
user_list = []
for riga in restaurant['user_id']:
  if riga not in listaVecchia:
    user_list.append(riga)


#random_user = lista id utenti random (10% del totale)
#random_user = random.choices(user_list, k=300)
#cnt = 0
#Passo ogni id di random_user in downloadUser per scaricare le foto degli utenti
#for user_id in random_user:
  #id = '%s' % user_id
  #downloadUser(id,cnt)
  #print(cnt)
  #cnt = cnt + 1

In [0]:
#random_user = lista id utenti random (10% del totale)
#random_user = random.choices(user_list, k=50)

cnt = 0
foto_elaborate = 0
#Passo ogni id di random_user in downloadUser per scaricare le foto degli utenti
for user_id in user_list:
  id = '%s' % user_id
  downloadUser(id,cnt)
  print(cnt)
  cnt = cnt + 1

In [3]:
foto_drive = drive.ListFile({'q': "'1Y4kVSFpVyLUA6uIAE9pAN12U82KNRZ6-' in parents and trashed=false"}).GetList()
lista_utenti_drive = []
for file1 in foto_drive:
  id = file1['title']
  lista_utenti_drive.append(id[0:22])

seen = set()
duplicati = []
for x in lista_utenti_drive:
    if x not in seen:
      seen.add(x)
    else:
      duplicati.append(x)

print(len(lista_utenti_drive))
print(duplicati)  

2093
['-1qsCgSntypM0SP0w62Lpg', 'wlTd-6M1pkO7zs5CgopQOQ']
